In [1]:
import sys
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision

torch.backends.cudnn.benchmark=True

# Unet

In [2]:
class double_conv(nn.Module):
    ''' conv -> BN -> relu -> conv -> BN -> relu'''
    def __init__(self, in_ch, out_ch):
        super(double_conv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )
        
    def forward(self, x):
        x = self.conv(x)
        return x

class inconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(inconv, self).__init__()
        self.conv = double_conv(in_ch, out_ch)
    
    def forward(self, x):
        return self.conv(x)

class down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()
        self.mpconv = nn.Sequential(
            nn.MaxPool2d(2),
            double_conv(in_ch, out_ch)
        )
    
    def forward(self, x):
        return self.mpconv(x)

class up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=True):
        super(up, self).__init__()
        
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_ch//2, out_ch//2, stride=2)
        
        self.conv = double_conv(in_ch, out_ch)
    
    def forward(self, x1, x2):
        x1 = self.up(x1)
        diffX = x2.size()[2] - x1.size()[2]
        diffY = x2.size()[3] - x1.size()[3]
        
        x1 = F.pad(x1, (diffX//2, diffX - diffX//2,
                        diffY//2, diffY - diffY//2)
                  )
        
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

class outconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(outconv, self).__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, 1)

    def forward(self, x):
        x = self.conv(x)
        return x
    

class UNet(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(UNet, self).__init__()
        self.inc = inconv(n_channels, 64)
        self.down1 = down(64,  128) #x2
        self.down2 = down(128, 256) #x3
        self.down3 = down(256, 512) #x4
        self.down4 = down(512, 512) #x5
        self.up1   = up(1024,256)
        self.up2   = up(512,128)
        self.up3   = up(256,64)
        self.up4   = up(128,64)
        self.outc  = outconv(64, n_classes)
    
    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        
        x = self.up1(x5,x4) # (x5-512d + x4-512d  = 1024d--> 256d)
        x = self.up2(x,x3)  # (x-256d + x3 - 256d = 512d --> 128d)
        x = self.up3(x, x2) # (x-128d + x2 - 128d = 256d --> 64d)
        x = self.up4(x,x1)  # (x-64d  + x1 - 64d  = 128d --> 64d)
        x = self.outc(x)    # 64d --> n_classes_D
        
        return x

class KaggleDataset(Dataset):
    def __init__(self, datatype):
        self.datatype = datatype
    
    def size(self):
        return (1,1)
    
    def __len__(self):
        return 3
    
    def __getitem__(self, idx):
        X = np.random.random((3,256,256)).astype('f')
        Y = np.random.random((2,256,256)).astype('f')
        return X, Y
    

def train(net, criterion, optimizer, epochs, trainLoader):
    print ('Training has begun ...')
    for epoch in range(epochs):
        for i, data in enumerate(trainLoader):
            try:
                print ('Epoch : {0} || BatchID : '.format(epoch,i)) 
                X,Y = data
                print (X.size(), Y.size())
                optimizer.zero_grad()

                Y_   = net(X.cuda())
                print (Y_.size())
                loss = criterion(Y_, Y.cuda())
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                if i % 2000 == 1999:    # print every 2000 mini-batches
                    print('[%d, %5d] loss: %.3f' %
                          (epoch + 1, i + 1, running_loss / 2000))
                    running_loss = 0.0 
            except Exception:
                sys.exit(1)
            
            

In [4]:
if __name__ == "__main__":
    net       = UNet(3, 2).cuda()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0005)
    criterion = nn.BCELoss()
    
    trainDataset      = KaggleDataset('train')
    trainDataLoader   = torch.utils.data.DataLoader(
            trainDataset
            , batch_size=4,shuffle=True
            , num_workers=1, pin_memory=True)
    
    train(net, criterion, optimizer, 2, trainDataLoader)
    
    
    verbose = 0
    if verbose:
        y = net(torch.Tensor(np.random.random((1,3,256,256))))
        print (y.size())
        
        print (net)
        
        for param in net.parameters():
            print (param.size())

Training has begun ...
Epoch : 0 || BatchID : 
torch.Size([3, 3, 256, 256]) torch.Size([3, 1, 256, 256])


/home/strider/anaconda3/lib/python3.5/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


torch.Size([3, 2, 256, 256])


/home/strider/anaconda3/lib/python3.5/site-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([3, 1, 256, 256])) that is different to the input size (torch.Size([3, 2, 256, 256])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


SystemExit: 1

/home/strider/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# del net
# torch.cuda.empty_cache()

# Rough

In [ ]:
# class KaggleDataset(Dataset):
#     def __init__(self, datatype):
#         self.datatype = datatype
    
#     def size(self):
#         return (1,1)
    
#     def __len__(self):
#         return 3
    
#     def __getitem__(self, idx):
#         X = np.random.random((3,256,256)).astype('f')
#         Y = np.random.random((1,256,256)).astype('f')
#         print (type(X))
#         return X, Y

# trainDataset = KaggleDataset('train')
# a,b = trainDataset[1]
# print (a.shape, b.shape)
# dataLoader = torch.utils.data.DataLoader(trainDataset
#             , batch_size=4,
#             shuffle=True, num_workers=1, pin_memory=True)

In [ ]:
# print (len(trainDataset))

# for i, data in enumerate(dataLoader):
#     print (i)
#     X,Y = data
#     print (type(X), X.size())